In [ ]:
## Load useful packages
!pip install wget
from random import sample

import keras
# import os.path
from os import path
import h5py
import keras.backend as K
import numpy as np
import tensorflow as tf
import wget
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils.np_utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, losses
from tensorflow.keras.preprocessing import image        


In [ ]:
classes = ('cloudy','rain','shine','sunrise')
data_path = 'https://raw.githubusercontent.com/Shujaat123/Weather_Classification/master/dataset/'

flist = []
for fname in classes:
  filename = 'WeatherClassificationDB_'+fname+'.mat'
  if(path.exists(filename)):
    !rm $filename
    print('existing file:', filename, ' has been deleted')
  print('downloading latest version of file:', filename)
  file_path = data_path + filename
  wget.download(file_path, filename)
  print('DONE')
  flist.append(filename)

In [ ]:
# Reading input and labels
cloudy_imgs = np.array(h5py.File(flist[0], 'r')['images']['input'])
cloudy_labels = np.array(h5py.File(flist[0], 'r')['images']['label'])

rain_imgs = np.array(h5py.File(flist[1], 'r')['images']['input'])
rain_labels = np.array(h5py.File(flist[1], 'r')['images']['label'])

shine_imgs = np.array(h5py.File(flist[2], 'r')['images']['input'])
shine_labels = np.array(h5py.File(flist[2], 'r')['images']['label'])

sunrise_imgs = np.array(h5py.File(flist[3], 'r')['images']['input'])
sunrise_labels = np.array(h5py.File(flist[3], 'r')['images']['label'])

#Setting labels from 0 to 3 (Intially: 1 to 4)
super_threshold_indices_CL = cloudy_labels > 0
cloudy_labels[super_threshold_indices_CL] = 0

super_threshold_indices_RA = rain_labels > 0
rain_labels[super_threshold_indices_RA] = 1

super_threshold_indices_SH = shine_labels > 0
shine_labels[super_threshold_indices_SH] = 2

super_threshold_indices_SUN = sunrise_labels > 0
sunrise_labels[super_threshold_indices_SUN] = 3

InputImages = np.concatenate((cloudy_imgs,rain_imgs,shine_imgs,sunrise_imgs), axis = 0)
InputImages = InputImages/InputImages.max()
ClassLabels = np.concatenate((cloudy_labels,rain_labels,shine_labels,sunrise_labels), axis = 0)

print(cloudy_imgs.shape)
print(rain_imgs.shape)
print(shine_imgs.shape)
print(sunrise_imgs.shape)
print(InputImages.shape)

In [ ]:
train_percent = 0.8
valid_percent = 0.1
num_CL_Tr = np.round(len(cloudy_imgs)*train_percent).__int__()
num_RA_Tr = np.round(len(rain_imgs)*train_percent).__int__()
num_SH_Tr = np.round(len(shine_imgs)*train_percent).__int__()
num_SUN_Tr = np.round(len(sunrise_imgs)*train_percent).__int__()

num_CL_Te = np.round(len(cloudy_imgs)*valid_percent).__int__()
num_RA_Te = np.round(len(rain_imgs)*valid_percent).__int__()
num_SH_Te = np.round(len(shine_imgs)*valid_percent).__int__()
num_SUN_Te = np.round(len(sunrise_imgs)*valid_percent).__int__()

print('Number of Training samples for cloudy images:',num_CL_Tr)
print('Number of Training samples for rainy images:',num_RA_Tr)
print('Number of Training samples for shiny images:',num_SH_Tr)
print('Number of Training samples for sunny images:',num_SUN_Tr)
CL_list = list(range(0,len(cloudy_imgs)))
RA_list = list(range(0,len(rain_imgs)))
SH_list = list(range(0,len(shine_imgs)))
SUN_list = list(range(0,len(sunrise_imgs)))
total_list  = CL_list+RA_list+SH_list+SUN_list

CL_Train = sample(CL_list,num_CL_Tr)
RA_Train = sample(RA_list,num_RA_Tr)
SH_Train = sample(SH_list,num_SH_Tr)
SUN_Train = sample(SUN_list,num_SUN_Tr)
train_list = CL_Train + RA_Train + SH_Train + SUN_Train

CL_Labels = cloudy_labels[CL_Train]
RA_Labels = rain_labels[RA_Train]
SH_Labels = shine_labels[SH_Train]
SUN_Labels = sunrise_labels[SUN_Train]
Label_train = np.concatenate((CL_Labels,RA_Labels,SH_Labels,SUN_Labels), axis = 0)


print('Number of Testing samples for cloudy images:',num_CL_Te)
print('Number of Testing samples for rainy images:',num_RA_Te)
print('Number of Testing samples for shiny images:',num_SH_Te)
print('Number of Testing samples for sunny images:',num_SUN_Te)

CL_Test = sample(set(CL_list) - set(CL_Train), num_CL_Te)
RA_Test = sample(set(RA_list) - set(RA_Train), num_RA_Te)
SH_Test = sample(set(SH_list) - set(SH_Train), num_SH_Te)
SUN_Test = sample(set(SUN_list) - set(SUN_Train), num_SUN_Te)
test_list = CL_Test + RA_Test + SH_Test + SUN_Test

CL_Labels_te = cloudy_labels[CL_Test]
RA_Labels_te = rain_labels[RA_Test]
SH_Labels_te = shine_labels[SH_Test]
SUN_Labels_te = sunrise_labels[SUN_Test]
Label_test = np.concatenate((CL_Labels_te,RA_Labels_te,SH_Labels_te,SUN_Labels_te), axis = 0)

CL_val = list(set(CL_list) - set(CL_Train) - set(CL_Test))
RA_val = list(set(RA_list) - set(RA_Train) - set(RA_Test))
SH_val = list(set(SH_list) - set(SH_Train) - set(SH_Test))
SUN_val = list(set(SUN_list) - set(SUN_Train) - set(SUN_Test))
val_list = CL_val+RA_val+SH_val+SUN_val

CL_Labels_val = cloudy_labels[CL_val]
RA_Labels_val = rain_labels[RA_val]
SH_Labels_val = shine_labels[SH_val]
SUN_Labels_val = sunrise_labels[SUN_val]
Label_val = np.concatenate((CL_Labels_val,RA_Labels_val,SH_Labels_val,SUN_Labels_val), axis = 0)

print('Number of Validation samples for cloudy images:',len(CL_Labels_val))
print('Number of Validation samples for rainy images:',len(RA_Labels_val))
print('Number of Validation samples for shiny images:',len(SH_Labels_val))
print('Number of Validation samples for sunny images:',len(SUN_Labels_val))


In [ ]:
#Setting Labels to categorical (One hot coding)
Label_train = np.squeeze(Label_train)
Label_train = to_categorical(Label_train)
print(Label_train.shape)

Label_test = np.squeeze(Label_test)
Label_test = to_categorical(Label_test)
print(Label_test.shape)

Label_val = np.squeeze(Label_val)
Label_val = to_categorical(Label_val)
print(Label_val.shape)

In [ ]:
#Concatenate train, test and validation Input images
Input_train = np.concatenate((cloudy_imgs[CL_Train],rain_imgs[RA_Train],shine_imgs[SH_Train],sunrise_imgs[SUN_Train]), axis = 0)
print(Input_train.shape)

Input_test = np.concatenate((cloudy_imgs[CL_Test],rain_imgs[RA_Test],shine_imgs[SH_Test],sunrise_imgs[SUN_Test]), axis = 0)
print(Input_test.shape)

Input_val = np.concatenate((cloudy_imgs[CL_val],rain_imgs[RA_val],shine_imgs[SH_val],sunrise_imgs[SUN_val]), axis = 0)
print(Input_val.shape)

In [ ]:
#Normalization between 0 and 1 
Input_train, Input_test, Input_val = Input_train / 255.0, Input_test / 255.0, Input_val/ 255.0

In [ ]:
#Pre-Trained Model VGG
# def new_model():
base_model = VGG16(include_top=False, weights = None, input_shape=(256,256,3), pooling='None')
for layer in base_model.layers:
        layer.trainable = False
  # add new classifier layers
flat1 = Flatten()(base_model.layers[-1].output)
flat1 = BatchNormalization()(flat1)
flat1 = Dropout(0.5)(flat1)
class1 = Dense(10, activation='relu')(flat1)
class2 = Dense(10, activation='relu')(class1)
output = Dense(4, activation='softmax')(class2)
  # define new model
Arch_pre = Model(inputs=base_model.inputs, outputs=output)
  # summarize
Arch_pre.summary()
opt = keras.optimizers.Adam(learning_rate=5e-4)
Arch_pre.compile(loss='categorical_crossentropy',metrics = ['accuracy'],optimizer=opt)
#     return my_model

# Arch_pre = new_model() 
len(base_model.trainable_variables)

In [ ]:
#Using the best model 
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, mode='min')

checkpoint = ModelCheckpoint('models\\modelweather-best.h5',
                                  verbose=0, monitor='val_loss',save_best_only=True, mode='auto')

Arch_pre.fit(Input_train, Label_train,
                epochs=200,
                batch_size=132,
                shuffle=True,
                validation_data=(Input_val, Label_val),
                callbacks = [es, checkpoint]
                )

del Arch_pre  # deletes the existing model
Arch_pre = load_model('models\\modelweather-best.h5')

In [ ]:
#Performance Evaluation on test data
Predicted_test =  Arch_pre.evaluate(Input_test,Label_test)
Predicted_test =  np.clip(Predicted_test, 0., 1.)